# ElectrodeSOHComposite issue

ElectrodeSOHComposite is not robust enough when using a composite electrode with hysteresis

In [1]:
import re
from itertools import product
from typing import Literal

import numpy as np
import tqdm

import pybamm

In [2]:
cell_name = "Chen2020_composite"
user_options = {
    "particle phases": ("2", "1"),
    "open-circuit potential": (("single", "one-state hysteresis"), "single"),
}
parameter_values = pybamm.ParameterValues(cell_name)

# Solving ESOH with the original Chen2020_composite parameters gives a 0% SOC
# voltage of 2.53V not 2.5V. We fix this by reducing the secondary initial
# concentration, which adjusts Q_Li to make the system consistent.
print(
    "Old c_s_init:",
    parameter_values[
        "Secondary: Initial concentration in negative electrode [mol.m-3]"
    ],
)
parameter_values.update(
    {"Secondary: Initial concentration in negative electrode [mol.m-3]": 2.3512e05}
)
print(
    "New c_s_init:",
    parameter_values[
        "Secondary: Initial concentration in negative electrode [mol.m-3]"
    ],
)

model = pybamm.lithium_ion.SPMe(user_options)

decay_rate = 100
parameter_values.update(
    {
        "Secondary: Negative particle hysteresis decay rate": decay_rate,
        "Secondary: Negative particle lithiation hysteresis decay rate": decay_rate,
        "Secondary: Negative particle delithiation hysteresis decay rate": decay_rate,
        "Secondary: Initial hysteresis state in negative electrode": 0,
    },
    check_already_exists=False,
)

inputs = {}
n_points = 101
showprogress = True

V_low_cut = parameter_values["Lower voltage cut-off [V]"]
V_high_cut = parameter_values["Upper voltage cut-off [V]"]

Old c_s_init: 276610.0
New c_s_init: 235120.0


/Users/robertwtimms/Documents/PyBaMM/src/pybamm/models/full_battery_models/lithium_ion/base_lithium_ion_model.py:279: UserWarning: The definition of the hysteresis decay rate parameter has changed in PyBaMM v25.10. Please see the CHANGELOG for more details.
  self.submodels[f"{domain} {phase} open-circuit potential"] = ocp_model(


Implement some helpers

In [3]:
def simulate(
    initial_value: float | str, direction: Literal["discharge", "charge"] | None = None
):
    """Create and run a PyBaMM simulation"""
    V_cut_off = V_high_cut if direction == "charge" else V_low_cut
    experiment = pybamm.Experiment(
        [
            (
                "Rest for 2 h",
                f"{direction.capitalize() if direction is not None else 'Discharge'} at C/30 until {V_cut_off} V",
            )
        ]
    )
    sim = pybamm.Simulation(
        model=model, parameter_values=parameter_values, experiment=experiment
    )
    sim.set_initial_state(initial_value, direction=direction, inputs=inputs)
    # sim.solve()
    return sim


def show_failures(failures: dict[str, Exception], n_max: int = 10) -> None:
    """Pretty print failing runs."""
    n_failures = len(failures)
    print(f"Number of failures: {n_failures}")
    for i, (run_id, exc) in enumerate(
        sorted(failures.items(), key=lambda item: item[0])
    ):
        if i >= n_max:
            break
        print(f"- {run_id}: {exc}")
    if n_failures > n_max:
        print(f"- And {n_failures - n_max} more...")


def get_average_error(failures: dict[str, Exception]) -> tuple[float | None, set[str]]:
    """Get the average error of the esoh solver."""
    number_pattern = r"[+-]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?"
    pattern = re.compile(
        r"Could not find acceptable solution: solver terminated unsuccessfully and maximum solution error "
        rf"\(({number_pattern})\) above tolerance \(({number_pattern})\)"
    )
    non_matching = set()
    errs = []
    for run_id, exc in failures.items():
        exc_match = pattern.fullmatch(str(exc))
        if exc_match is None:
            non_matching.add(run_id)
        else:
            errs.append(float(exc_match.group(1)))
    err_avg = np.mean(errs) if errs else None
    return err_avg, non_matching

## Case 1: Initialization method - SoC

In [4]:
directions = ["discharge", "charge", None]
initial_values = np.linspace(0, 1, n_points).tolist()

pairs = list(product(directions, initial_values))
if showprogress:
    pairs = tqdm.tqdm(pairs, total=len(pairs), desc="Simulations", unit="run")

failures_soc = {}
for direction, initial_value in pairs:
    run_id = f"{cell_name}_dir_{direction}_soc_ini_{initial_value:.6g}"
    try:
        sim = simulate(initial_value, direction)
    except Exception as e:
        failures_soc[run_id] = e

Simulations: 100%|██████████| 303/303 [00:11<00:00, 26.22run/s]


In [5]:
# Overall results
print(f"Overall success ratio: {1 - len(failures_soc) / len(pairs):.2%}")

# Split by direction
print("\n" + "=" * 60)
print("SUCCESS/FAILURE SPLIT BY DIRECTION")
print("=" * 60)

for direction in ["discharge", "charge", None]:
    dir_str = str(direction)
    dir_pairs = [
        p for p in list(product(directions, initial_values)) if p[0] == direction
    ]
    dir_failures = {k: v for k, v in failures_soc.items() if f"_dir_{dir_str}_" in k}
    n_total = len(dir_pairs)
    n_failures = len(dir_failures)
    n_success = n_total - n_failures
    success_rate = n_success / n_total if n_total > 0 else 0

    # Get average error for this direction
    avg_err, _ = get_average_error(dir_failures)
    avg_err_str = f"{avg_err:.4f} V" if avg_err else "N/A"

    print(f"\nDirection: {dir_str}")
    print(f"  Success: {n_success}/{n_total} ({success_rate:.1%})")
    print(f"  Failures: {n_failures}")
    print(f"  Average residual: {avg_err_str}")

print("\n" + "=" * 60)
print("\nSample failures:")
show_failures(failures_soc)

Overall success ratio: 100.00%

SUCCESS/FAILURE SPLIT BY DIRECTION

Direction: discharge
  Success: 101/101 (100.0%)
  Failures: 0
  Average residual: N/A

Direction: charge
  Success: 101/101 (100.0%)
  Failures: 0
  Average residual: N/A

Direction: None
  Success: 101/101 (100.0%)
  Failures: 0
  Average residual: N/A


Sample failures:
Number of failures: 0


## Case 2: Initialization method - Voltage

In [6]:
directions = ["discharge", "charge", None]
initial_values = [f"{v:.6g} V" for v in np.linspace(V_low_cut, V_high_cut, n_points)]

pairs = list(product(directions, initial_values))
if showprogress:
    pairs = tqdm.tqdm(pairs, total=len(pairs), desc="Simulations", unit="run")

failures_voltage = {}
for direction, initial_value in pairs:
    run_id = f"{cell_name}_dir_{direction}_v_ini_{initial_value}"
    try:
        sim = simulate(initial_value, direction)
    except Exception as e:
        failures_voltage[run_id] = e

Simulations: 100%|██████████| 303/303 [00:13<00:00, 22.27run/s]


In [7]:
# Overall results
print(f"Overall success ratio: {1 - len(failures_voltage) / len(pairs):.2%}")

# Split by direction
print("\n" + "=" * 60)
print("SUCCESS/FAILURE SPLIT BY DIRECTION")
print("=" * 60)

for direction in ["discharge", "charge", None]:
    dir_str = str(direction)
    dir_pairs = [
        p for p in list(product(directions, initial_values)) if p[0] == direction
    ]
    dir_failures = {
        k: v for k, v in failures_voltage.items() if f"_dir_{dir_str}_" in k
    }
    n_total = len(dir_pairs)
    n_failures = len(dir_failures)
    n_success = n_total - n_failures
    success_rate = n_success / n_total if n_total > 0 else 0

    # Get average error for this direction
    avg_err, _ = get_average_error(dir_failures)
    avg_err_str = f"{avg_err:.4f} V" if avg_err else "N/A"

    print(f"\nDirection: {dir_str}")
    print(f"  Success: {n_success}/{n_total} ({success_rate:.1%})")
    print(f"  Failures: {n_failures}")
    print(f"  Average residual: {avg_err_str}")

print("\n" + "=" * 60)
print("\nSample failures:")
show_failures(failures_voltage)

Overall success ratio: 95.05%

SUCCESS/FAILURE SPLIT BY DIRECTION

Direction: discharge
  Success: 96/101 (95.0%)
  Failures: 5
  Average residual: 0.0663 V

Direction: charge
  Success: 96/101 (95.0%)
  Failures: 5
  Average residual: 0.0663 V

Direction: None
  Success: 96/101 (95.0%)
  Failures: 5
  Average residual: 0.0663 V


Sample failures:
Number of failures: 15
- Chen2020_composite_dir_None_v_ini_2.517 V: Could not find acceptable solution: solver terminated unsuccessfully and maximum solution error (0.02813882348906649) above tolerance (1e-06)
- Chen2020_composite_dir_None_v_ini_2.534 V: Could not find acceptable solution: solver terminated unsuccessfully and maximum solution error (0.011138823489064809) above tolerance (1e-06)
- Chen2020_composite_dir_None_v_ini_2.568 V: Could not find acceptable solution: solver terminated unsuccessfully and maximum solution error (0.0068197174846362785) above tolerance (1e-06)
- Chen2020_composite_dir_None_v_ini_2.585 V: Could not find acc